In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
%%writefile huffman_encoding.cu
#include <stdio.h>
#include <cuda.h>

__global__ void encodeHuffman(char *input, char *output, int *codes) {
    int idx = threadIdx.x;
    output[idx] = codes[input[idx]];
}

int main() {
    char h_input[] = "HELLOCUDA";
    char h_output[10];
    int h_codes[128];  // Dummy Huffman codes

    for (int i = 0; i < 128; i++) h_codes[i] = i % 256;

    char *d_input, *d_output;
    int *d_codes;
    cudaMalloc((void**)&d_input, 10 * sizeof(char));
    cudaMalloc((void**)&d_output, 10 * sizeof(char));
    cudaMalloc((void**)&d_codes, 128 * sizeof(int));

    cudaMemcpy(d_input, h_input, 10 * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_codes, h_codes, 128 * sizeof(int), cudaMemcpyHostToDevice);

    encodeHuffman<<<1, 10>>>(d_input, d_output, d_codes);

    cudaMemcpy(h_output, d_output, 10 * sizeof(char), cudaMemcpyDeviceToHost);

    printf("Encoded: %s\n", h_output);

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_codes);
    return 0;
}


Writing huffman_encoding.cu


In [4]:
!nvcc huffman_encoding.cu -o huffman
!./huffman


Encoded: HELLOCUDA
